- To rerun any S-LOCAL network you need a minimum of 32GB of RAM/VRAM memory on the device used for training
- Average time to run a 100 epoch on a Nvidia Tesla V100 GPU is 20 minutes
- A faster implementation of the Locally Connected Layer is available in the Keras API : 
- https://keras.io/api/layers/locally_connected_layers/locall_connected2d/

## Imports

In [1]:
INIT_PATH = "."
DOWNLOAD_DATA = False

# To use google colab processing uncomment following lines

#INIT_PATH = "MyDrive/inference_and_learning"
#from google.colab import drive
#MOUNT_PATH = "/content/drive"
#drive.mount(MOUNT_PATH)
#INIT_PATH = MOUNT_PATH + "/" + INIT_PATH


# To train LOCAL change to True
LOCAL_TRAIN = False 

In [2]:
import os.path,sys
import networks as nnets
import pandas as pd, torch
from typing import Tuple, List
import torchvision
import torch
import json
torch.set_default_dtype(torch.float16)
sys.path.append(INIT_PATH)
torch.set_default_dtype(torch.float16)

In [3]:
DATA_PATH = "." # Path to data folder
if (DOWNLOAD_DATA):
    torchvision.datasets.CIFAR10(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.CIFAR100(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True)

    torchvision.datasets.CIFAR10(root=DATA_PATH +'/data', download=True,train = False)
    torchvision.datasets.CIFAR100(root=DATA_PATH +'/data', download=True ,train = False) 
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True, split = "test")


## OPTIONAL: optimization of parameters

You may use this script to test different hyperparameters
Load the network description in the file files/optim.json
The file is a json file with the following structure :
```json
{
    "optim":{ // Hyperparameters to optimize
        "model":{ // Hyperparameters to optimize for the model
            "S-LOCAL":{ // Model you want to optimize hyperparameters on 
                "hidden_size":[64,128,256,512], // List of values to test
                "dropout":[0.1,0.2,0.3,0.4,0.5] // List of values to test
            }
        }
    },
    "networks":{ // List of networks to test
        "S-LOCAL-64-0.1":{ // Name of the network
            //add the network default description here
        }
    },
    "results":{ // Results of the optimization
        "S-LOCAL-64-0.1":{ //Name of the network
            "best":{ 
                // Best hyperparameters found
            },
            "accuracy":0.9 // Accuracy of the best hyperparameters
        }
    }
}
```

In [4]:
optim = json.load(open(INIT_PATH+"/files/optim.json"))
optimer = nnets.NetworkOptim(optim["optim"],DATA_PATH,epoch=100,verbose = True,very_verbose = True)

for key,item in optim["networks"].items():
    if key in optim["results"]:
        print(f"Skipping {key} as it is already done")
        continue
    if not LOCAL_TRAIN and item["model"]["name"] in ["S-LOCAL","D-LOCAL"]:
        print(f"Skipping {key} as it is a localy connected network")
        continue
    print(f"Starting {item['model']['name']} with {item['optimizer']['name']} on {item['dataset']['name']}")
    acc,best = optimer(item)
    print(f"Best : {best} with accuracy {acc}")
    optim["results"][key] = {"best":best,"accuracy":acc}
    json.dump(optim,open(INIT_PATH+"/files/optim.json","w"))

Starting S-CONV with SGD on CIFAR-10
Calculating accuracy for default parameters
batch_size : 512
epoch      : 100
use_cuda   : True
optimizer :
	name         : SGD
	lr           : 0.1
	momentum     : 0.9
	weight_decay : 0
dataset :
	name       : CIFAR-10
	transforms :
		0 : ToTensor
model :
	name      : S-CONV
	alpha     : 204
	dropout_1 : 0
	dropout_2 : 0.05
scheduler :
	name       : CosineAnnealingWarmRestarts
	T_0        : 20
	T_mult     : 2
	eta_min    : 0
	last_epoch : -1
Training Network: S-CONV, CIFAR-10, SGD using : 255.802546 M parameters on cuda
36.90%, 46.80%, 51.24%, 53.63%, 55.75%, 58.23%, 60.20%, 61.58%, 62.29%, 63.83%,  : 10/100
62.85%, 64.37%, 63.84%, 64.54%, 65.20%, 64.76%, 65.26%, 65.30%, 65.09%, 64.71%,  : 20/100
59.10%, 60.41%, 62.27%, 63.43%, 64.32%, 64.50%, 64.69%, 64.32%, 64.22%, 64.68%,  : 30/100
64.72%, 64.69%, 64.62%, 64.47%, 64.94%, 64.91%, 64.87%, 64.27%, 64.75%, 64.71%,  : 40/100
64.56%, 64.90%, 64.65%, 64.51%, 64.86%, 64.67%, 65.02%, 64.97%, 64.86%, 64.89

FileNotFoundError: [Errno 2] No such file or directory: './network_description/optim.json'

## Reproducing the paper

### Table 2


In [ ]:
table2 = pd.read_csv(INIT_PATH+"/files/table2.csv")
table2_net = json.load(open(INIT_PATH+"/files/table2.json"))
RETRAIN = True

if RETRAIN :
    for dataset in ["CIFAR-10","CIFAR-100","SVHN"]:
        for l in range(len(table2)):
            network = table2.iloc[l].to_dict()
            if network["train"]=="False" or (network[dataset] == "NaN"):
                continue
            ID = "_".join([network[i] for i in ["Model","Training_Method"]]+[dataset])
            if not LOCAL_TRAIN and table2_net[ID]["model"]["name"] in ["S-LOCAL","D-LOCAL"]:
                print(f"Skipping {key} as it is a localy connected network")
                continue
            save_model_0 = False
            if network["model"] == "S-FC" and network["training_method"] == "B-lasso(B=50)":
                save_model_0 = True
            acc,model,non_zero = nnets.train(table2_net[ID],DATA_PATH,verbose = True,save_number_of_non_zero=save_model_0,very_verbose=False) 
            table2.loc[l,dataset] = acc
            table2.to_csv(INIT_PATH+"/csv/table2.csv",index = False)
            torch.save(model.state_dict(),INIT_PATH+"/network_saved/"+ID+".pt")
            if(save_model_0):
                json.dump(non_zero,open(INIT_PATH+"/files/figure3.json","w"))

print(table2)

### figure 3
We can reuse result saved while training table 2

### figure 4